<h1>Emotion Analysis - Part 1</h1>
<h2><i>Data Preparation and Emotion Analysis with Text-Classification Model</i></h2>

<h3>[1.1][Imports]</h3>

In [ ]:
### Imports ###

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import style

style.use('ggplot')
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
import json
import torch
from transformers import pipeline
from tqdm import tqdm
import pytz
from scipy import stats
from scipy.stats import f_oneway
import re
from nltk.stem.porter import PorterStemmer

### Set necessary workspace variables ###

# Set execution type (to avoid repeating resource intensive operations)
#RUN_TYPE = 0 # set to 0 to avoid file creation process
RUN_TYPE = 1 # set to 1 to perform emotion analysis file creation process
#RUN_TYPE = 2 # set to 2 to perform topic analysis file creation process

# Define Ekman's emotions 
ekman_emotions = ['anger', 'fear', 'joy', 'sadness', 'disgust', 'surprise']

# Timezones
eastern = pytz.timezone("US/Eastern")
european = pytz.timezone("Europe/Berlin")

<h3>[1.2][Data Preparation & Cleaning]</h3>
<p>Normalizing the quoted_tweets-dictionary to create a dataframe that contains the text of the quoted and origional tweet.</p>
<p>The following cell contains functions for the second step of data preparation, namely removing noise, truncating text, stemming words, and subsequently performing the emotion analysis.</p>

In [ ]:
### Process File with Tweet-Data (load and extract necessary columns) ### 
## Import dataset
df_tweets = pd.read_csv(
    '../data/twitter/tweets_isTweet.csv',
    dtype={'id': 'object'},
    low_memory=False
)
## Parameter for weighing emotions per event later in Event Study
df_tweets['total_engagement'] = (df_tweets['retweetCount'] +
                                 df_tweets['likeCount'] +
                                 df_tweets['replyCount'] +
                                 df_tweets['quoteCount'] +
                                 df_tweets['viewCount'])
df_tweets = df_tweets[['id', 'createdAt', 'text', 'quoted_tweet', 'total_engagement']]
df_tweets = df_tweets.rename(columns={'id': 'tweet_id', 'text': 'tweet_text'})

## Normalize json column
quoted_tweets_normalized = pd.json_normalize(
    df_tweets['quoted_tweet'].apply(
        lambda x: json.loads(x) if pd.notna(x) and isinstance(x, str) else None
    )
)
quoted_tweets_normalized = quoted_tweets_normalized.rename(columns={
    'id': 'quoted_tweet_id',
    'text': 'quoted_tweet_text'
})

## Link by index
df_tweets.index = quoted_tweets_normalized.index

## Concat both dataframes
df_tweets_normalized = pd.concat([
    df_tweets[['tweet_id', 'createdAt', 'tweet_text', 'total_engagement']],
    quoted_tweets_normalized[['quoted_tweet_id', 'quoted_tweet_text']]
], axis=1)

## View data
print(df_tweets.columns)
#df_tweets_normalized.head()

In [ ]:
### Prepare data for analysis (append emotions and topics) ###

# 1. Set Classifier
# Load Hugging Face's emotion classifier
print("[Info]")
device = "cuda" if torch.cuda.is_available() else "cpu"
classifier = pipeline("text-classification", model="bhadresh-savani/bert-base-uncased-emotion", top_k=None, device=0 if device == "cuda" else -1)

# 2. Functions to clean and process text data 
# Removing noise from the text
def remove_noise(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+httpss\S+", '', text, flags=re.MULTILINE) # Remove Url
    text = re.sub(r"\@w+|\#", '', text) # remove @ and #
    text = re.sub(r"[^\w\s]", '', text) # remove punctuation
    text_tokens = text.split()
    filtered_text = [w for w in text_tokens if not w in stop_words] # remove stopwords
    return " ".join(filtered_text)

# Reduction of dimensionality by abstracting word to word stem and truncating text
stemmer = PorterStemmer()
def stem_words(text):
    words = text.split()
    stemmed_text = [stemmer.stem(word) for word in words] # abstract to word stem
    return stemmed_text

def truncate_text(text, max_length=512):
    words = text.split()
    return " ".join(words[:max_length])

<h4>[1.2.1] Append Emotions</h4>

In [ ]:
# 3. Computing and appending emotions to dataframe
def compute_emotions(text):
    if not isinstance(text, str) or text.strip() == "":
        print("[ComputeEmotions] Empty cell after data cleaning. Returning 0.0 for all emotions.")
        return {emotion: 0.0 for emotion in ekman_emotions}

    try:
        # Classify emotions using the Hugging Face pipeline and handle errors
        results = classifier(text)[0]
        if not results or not isinstance(results, list) or len(results[0]) == 0:
            return {emotion: 0.0 for emotion in ekman_emotions}

        emotion_scores = {result['label']: result['score'] for result in results}
        return {emotion: emotion_scores.get(emotion, 0.0) for emotion in ekman_emotions}

    except Exception as e:
        print(f"[ComputeEmotions] Error while processing text: {text[:20]}... Error: {e}")
        return {emotion: 0.0 for emotion in ekman_emotions}

def append_emotions(df: pd.DataFrame, text_column: str) -> pd.DataFrame:
    if text_column not in df.columns:
        raise ValueError(f"[AppendEmotions] Column '{text_column}' not found in DataFrame.")
    
    print("[AppendEmotions] Computing emotions for column:", text_column)

    cleaned_column = f"{text_column}_cleaned"
    df[cleaned_column] = df[text_column].apply(
        lambda x: " ".join(stem_words(remove_noise(x))) if isinstance(x, str) and x.strip() else ""
    )

    # Truncate text if cleaned text exceeds 512 tokens
    if (df[cleaned_column].str.split().str.len() > 512).any():
        print("[AppendEmotions] At least one row with more than 512 tokens - truncating text ...")
        df[cleaned_column] = df[cleaned_column].apply(lambda x: truncate_text(x, max_length=512))

    emotion_scores = [compute_emotions(text) for text in tqdm(df[cleaned_column], desc="[AppendEmotions] Processing emotions")]
    emotions_df = pd.DataFrame(emotion_scores)
    emotions_df.index = df.index
    emotions_df.columns = [f"{text_column}_{emotion}" for emotion in ekman_emotions]
    
    # Add dominant emotion column
    dominant = emotions_df.idxmax(axis=1).apply(lambda x: x.split('_')[-1])
    all_zero = emotions_df.eq(0.0).all(axis=1)
    dominant[all_zero] = np.nan
    emotions_df[f"{text_column}_dominant_emotion"] = dominant

    # Insert right hand of input text_column
    insert_at = df.columns.get_loc(text_column) + 1

    # DataFrame in drei Teile splitten und zusammenfügen
    left = df.iloc[:, :insert_at]
    right = df.iloc[:, insert_at:].drop(columns=[cleaned_column], errors='ignore')
    result_df = pd.concat([left, df[[cleaned_column]], emotions_df, right], axis=1)

    return result_df

<p>This part is only run once to create the new csv-file. Subsequently, the further analysis is performed on the new dataset.</p>

In [ ]:
### Create Datasets: Performing Emotion and Content Analysis ###

# Perform emotion analysis for columns specified and safe as new csv-file
if RUN_TYPE == 1:
    for col in ['tweet_text']:#, 'quoted_tweet_text']:
        df_tweets_normalized = append_emotions(df_tweets_normalized, text_column=col)
        
    df_tweets_normalized.to_csv('../data/twitter/tweets_isTweet_emotions.csv', index=False)

<h1>Emotion Analysis - Part 2</h1>
<h2><i>Formatting & Data Exploration</i></h2>

<h3>[2.1][Data Formatting]</h3>

In [ ]:
### Read necessary datasets (tweets & financial) and convert to the same timezone ###

# Set timezone 
eastern = pytz.timezone("US/Eastern")

# Tweet data
df_tweets_normalized = pd.read_csv('../data/twitter/tweets_isTweet_emotions.csv')[
    ['tweet_id',
     'createdAt',
     'total_engagement',
     'tweet_text_anger',
     'tweet_text_fear',
     'tweet_text_joy',
     'tweet_text_sadness',
     'tweet_text_disgust',
     'tweet_text_surprise',
     'tweet_text_dominant_emotion'
    ]
].dropna()

df_tweets_normalized = df_tweets_normalized.rename(
    columns={
        'tweet_id': 'id',
        'total_engagement': 'engagement',
        'createdAt': 'timestamp',
        'tweet_text_anger': 'anger',
        'tweet_text_fear': 'fear',
        'tweet_text_joy': 'joy',
        'tweet_text_sadness': 'sadness',
        'tweet_text_disgust': 'disgust',
        'tweet_text_surprise': 'surprise',
        'tweet_text_dominant_emotion': 'tweet_emotion'
    }
)

df_tweets_normalized['timestamp'] = pd.to_datetime(
    df_tweets_normalized['timestamp'],
    format="%a %b %d %H:%M:%S %z %Y",
    errors="coerce"
)

df_tweets_normalized['timestamp'] = df_tweets_normalized['timestamp'].dt.tz_convert(
    tz=eastern
)

# Stock data
df_stock_data = pd.read_csv('../legacy/data/tsla_intraday_202305_202504-1m.csv')

df_stock_data = df_stock_data.rename(
    columns={'Unnamed: 0': 'timestamp'}
)

df_stock_data['timestamp'] = pd.to_datetime(
    df_stock_data['timestamp']
).dt.tz_localize(
    tz=eastern
)

<h3>[2.2][Log Transformation]</h3>
<p>
    To analyse the fluctuations in return and volume, we use a logarithmized scale. This has some advantages:
    <!--
        Note for me:
        The return is the difference between prev and actual closing price:
        ln(actual/prev) = ln(actual) - ln(prev) // continuous returns are computed based on logarithmized price change
        For volume, we are interested in absolute values that where traded and not in relative changes, thus, we do not compare a actual and prev value. The volume can be 0 for some points in time, thus we must use log1p to cope with the case where log(0) would happen. Logarithmizing scales of the dispersed data to make it comparable.
        log1p(x) = log(1 + x) 
        => for small values of x, operates like a linear transformation
        => for large values of x, operates like a log transformation 
    -->
    <ul>
        <li>
            Stationary time-series: When observing the absolute trend, the price fluctuates and rises / falls over time (the mean changes over time). However, when looking at the changes per minute, the values fluctuate around a constant means.
        </li>
        <li>
            Additivity: By adding the log_returns (stationary values per minute), we can compute the overall effect within a time window. The single fluctuations around the constant mean add up to show a general fluctuation from a constant mean during the observed time window.
        </li>
        <li>
            Symmetry of up- and downwards trends: The return (relative change) is not symmetrical, the log_return is. In example:<br>
            return: 10 -> 11 = 1.1 ; 11 -> 10 = 0.91 <br>
            log_return: 10 -> 11 = 0.095 ; 11 -> 10 = -0.095
        </li>
        <li>Closer to being normally distributed (compared to absolute returns)
            <ul>
                <li>Central limit theorem: when adding many independent variables, their sum approximates a normal distribution</li>
                <li>Here: as trends are the sum of many independet factors, the sum of their deviation will approximate a normal distribution</li>
            </ul>
        </li>
    </ul>
</p>

In [ ]:
### Computing log return/volume and intraday profile ###

# Compute minute_of_day and intraday_profile for seasonality analysis
df_stock_data['minute_of_day'] = df_stock_data['timestamp'].dt.hour * 60 + df_stock_data['timestamp'].dt.minute

# Return
df_stock_data['log_return'] = np.log(
    df_stock_data['close'] / df_stock_data['close'].shift(1)
)

df_stock_data['log_return_z'] = (df_stock_data['log_return'] - df_stock_data['log_return'].mean()) / df_stock_data['log_return'].std()


# Volume (use natural logarithm with 1-shift to avoid log(0) issues - see Bamberg (1987)))
df_stock_data['log_volume'] = np.log1p( # Using log1p to avoid issues with log(0), i.e., when no trades were made
    df_stock_data['volume']             # => log1p(x) = log(x + 1)
)

intraday_profile = df_stock_data.groupby('minute_of_day')[['log_return', 'log_volume']].agg(['mean', 'std']).reset_index()
df_stock_data['log_volume_z_intraday'] = df_stock_data.groupby('minute_of_day')['log_volume'].transform(
    lambda x: (x - x.mean()) / x.std()
)
df_stock_data['log_return_z_intraday'] = df_stock_data.groupby('minute_of_day')['log_return'].transform(
    lambda x: (x - x.mean()) / x.std()
)

df_stock_data.head()

In [ ]:
# Test: Get median of log volume 

vol_list = df_stock_data['log_volume'].tolist()
median_log_volume = np.median(vol_list)
print(f"[Median Log Volume] Median of log volume: {median_log_volume:.3f}")

mean_log_volume = np.mean(vol_list)
print(f"[Mean Log Volume] Mean of log volume: {mean_log_volume:.3f}")

<h3>[2.3][Testing for Seasonality]</h3>
<p>
    Before analyzing whether or not we can observe a real effect of emotions on the stock, we must cancel out a possible bias, namely the intraday seasonality. The term describes the effect that the stock price / trading volume is influenced by the time of day, where the two metrics experience higher volatility at market opening and closing hours than during the day.
</p>
<p>
    To test for this effect, we must group the return / volume by the time of day and compare the means / variances of the groups. With the F-test we can check whether or not the mean values differ significantly between the different groups (here: hours of the day).
</p>
<p>
    To apply the F-test, we can use the f_oneway-Method, which tests the null hypothesis that two or more groups have the same population mean. The test is applied to samples of two or more groups, possibly with different sizes. Here, it is applied to the data grouped by hours of the day and test whether the different goups (hours) have the same population mean. If a group mean significantly differs from the other groups means, we can conclude intraday effects.
</p>

In [ ]:
### Plotting intraday profile of means ### 

print("[Intraday Profile] Visualization of intraday seasonality: Course of means per minute of day")

# Convert minute of the day to time_label 
intraday_profile['hour'] = intraday_profile['minute_of_day'] // 60
intraday_profile['minute'] = intraday_profile['minute_of_day'] % 60
intraday_profile['time_label'] = intraday_profile['hour'].astype(str) + ':' + intraday_profile['minute'].astype(str).str.zfill(2)

# Subplot with two y-axes for different scales
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# Plot 1: Log Return means
ax1.plot(intraday_profile['minute_of_day'], intraday_profile[('log_return', 'mean')], 
         color='blue', linewidth=2, label='Mean log_return')
ax1.axhline(y=0, color='red', linestyle='--', alpha=0.7, label='Origin (Zero)')
ax1.set_title('[Avg. log_volume per min]', fontsize=14, loc='left')
ax1.set_xlabel('Minute of the day')
ax1.set_ylabel('Avg. log_return')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Market hours of NY Stock Exchange in eastern timezone 
market_times = {
    570: '9:30 (Opening)',
    720: '12:00 (Noon)', 
    960: '16:00 (Closing)'
}

# Plot 2: Log Volume means
ax2.plot(intraday_profile['minute_of_day'], intraday_profile[('log_volume', 'mean')], 
         color='green', linewidth=2, label='Mean log_volume')
ax2.set_title('[Avg. log_volume per min]', fontsize=14, loc='left')
ax2.set_xlabel('Minute of the day')
ax2.set_ylabel('Avg. log_volume')
ax2.grid(True, alpha=0.3)
ax2.legend()

# Mark market hours in both plots
for ax in [ax1, ax2]:
    for minute, label in market_times.items():
        if minute in intraday_profile['minute_of_day'].values:
            ax.axvline(x=minute, color='gray', linestyle=':', alpha=0.6)
            y_pos = ax.get_ylim()[0] + (ax.get_ylim()[1] - ax.get_ylim()[0]) * 0.9
            ax.text(minute, y_pos, label, fontsize=8, ha='right')
            
plt.tight_layout()
plt.show()

# Stats
print(f"[Mean Stats] Intraday Profile of means of TSLA Stock Data")
print(f"Log Return:")
print(f"  - Min: {intraday_profile[('log_return', 'mean')].min():.6f}")
print(f"  - Max: {intraday_profile[('log_return', 'mean')].max():.6f}")
print(f"  - Span: {intraday_profile[('log_return', 'mean')].max() - intraday_profile[('log_return', 'mean')].min():.6f}")

print(f"Log Volume:")
print(f"  - Min: {intraday_profile[('log_volume', 'mean')].min():.3f}")
print(f"  - Max: {intraday_profile[('log_volume', 'mean')].max():.3f}")
print(f"  - Span: {intraday_profile[('log_volume', 'mean')].max() - intraday_profile[('log_volume', 'mean')].min():.3f}")

min_volume_time = intraday_profile.loc[intraday_profile[('log_volume', 'mean')].idxmin(), 'time_label']
max_volume_time = intraday_profile.loc[intraday_profile[('log_volume', 'mean')].idxmax(), 'time_label']

print(f"\n[Peak Times]")
print(f"Lowest volume at: {min_volume_time.iloc[0]}")
print(f"Highest volume at: {max_volume_time.iloc[0]}")

In [ ]:
### Plotting volatility (std) of log return / volume and perform F-test ###

print("[Intraday Profile] Visualization of intraday seasonality: Volatility (std) per minute of day")

# Subplot with two y-axes for different scales
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# Plot 1: Log Return volatility
ax1.plot(intraday_profile['minute_of_day'], intraday_profile[('log_return', 'std')], 
         color='red', linewidth=2, label='Volatility log_return (Std.)')
ax1.set_title('[Standard Deviation log_return per Minute]', fontsize=14, loc='left')
ax1.set_xlabel('Minute of the day')
ax1.set_ylabel('Std. log_return')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Plot 2: Log Volume Volatility
ax2.plot(intraday_profile['minute_of_day'], intraday_profile[('log_volume', 'std')], 
         color='orange', linewidth=2, label='Volatility log_volume (Std.)')
ax2.set_title('[Standard Deviation log_volume per Minute]', fontsize=14, loc='left')
ax2.set_xlabel('Minute of the day')
ax2.set_ylabel('Std. log_volume')
ax2.grid(True, alpha=0.3)
ax2.legend()

# Mark market hours in both plots
for ax in [ax1, ax2]:
    for minute, label in market_times.items():
        if minute in intraday_profile['minute_of_day'].values:
            ax.axvline(x=minute, color='gray', linestyle=':', alpha=0.6)
            y_pos = ax.get_ylim()[0] + (ax.get_ylim()[1] - ax.get_ylim()[0]) * 0.9
            ax.text(minute, y_pos, label, fontsize=8, ha='right')

plt.tight_layout()
plt.show()

# Volatility stats
print(f"[Volatility Stats] Intraday Profile of Volatility (Std.) of TSLA Stock Data")
print(f"Log Return Volatility:")
print(f"  - Minimum Std: {intraday_profile[('log_return', 'std')].min():.6f}")
print(f"  - Maximum Std: {intraday_profile[('log_return', 'std')].max():.6f}")
print(f"  - Proportion Max/Min: {intraday_profile[('log_return', 'std')].max() / intraday_profile[('log_return', 'std')].min():.2f}x")

print(f"\nLog Volume Volatility:")
print(f"  - Minimum Std: {intraday_profile[('log_volume', 'std')].min():.3f}")
print(f"  - Maximum Std: {intraday_profile[('log_volume', 'std')].max():.3f}")
print(f"  - Proportion Max/Min: {intraday_profile[('log_volume', 'std')].max() / intraday_profile[('log_volume', 'std')].min():.2f}x")

min_vol_return_time = intraday_profile.loc[intraday_profile[('log_return', 'std')].idxmin(), 'time_label']
max_vol_return_time = intraday_profile.loc[intraday_profile[('log_return', 'std')].idxmax(), 'time_label']

print(f"\n[Peak Times]")
print(f"Lowest Return-Volatility at: {min_vol_return_time.iloc[0]}")
print(f"Highest Return-Volatility at: {max_vol_return_time.iloc[0]}")

# Test for Seasonality
# Group data by hours and test for differences
df_stock_data['hour'] = df_stock_data['minute_of_day'] // 60
hourly_returns = [group['log_return'].dropna() for name, group in df_stock_data.groupby('hour')]
hourly_volumes = [group['log_volume'].dropna() for name, group in df_stock_data.groupby('hour')]

# F-test for equality of variances over the hours
if len(hourly_returns) > 2:  # At least 3 groups for F-Test, * marks a series of groups
    f_stat_returns, p_val_returns = f_oneway(*hourly_returns)
    f_stat_volumes, p_val_volumes = f_oneway(*hourly_volumes)
    
    print(f"\n[Statistical Test for Intraday Seasonality]")
    print(f"F-Test for log_return for hours: F={f_stat_returns:.3f}, p={p_val_returns:.6f}")
    print(f"F-Test for log_volume for hours: F={f_stat_volumes:.3f}, p={p_val_volumes:.6f}")
    
    alpha = 0.05
    print(f"\nWith α = {alpha}:")
    print(f"Detected significant intraday differences for log_returns: {'Yes' if p_val_returns < alpha else 'No'}")
    print(f"Detected significant intraday differences for log_volumes: {'Yes' if p_val_volumes < alpha else 'No'}")

<p><i>Intraday Seasonality Test Results</i></p>
<p>Log_return
    <ul>
        <li>Results: No significant intraday seasonality observable</li>
        <li>Implications: No need to incorporate the time of day in our analysis of the log return</li>
    </ul>
</p>
<p>Log_volume
    <ul>
        <li>Results: Significant intraday seasonality observable - time of the day has effect on trading volume</li>
        <li>Implications: Must include the effect into our analysis</li>
        <li>How: intraday z-normalization of log_volume</li>
    </ul>

<h3>[2.4][Normalization]</h3>
<p>
    To evaluate whether an event has a significant effect on the return / volume, we must check if the mean from the event-window (point in time or timeframe) significantly differentiates from the mean of the estimation window, which we use as baseline for comparison. This can be done by performing a t-test, however, this presumes the variable to be normally distributed.
</p>
<p>
    By plotting the log_return and log_volume we can see, that the log_return already is approximately normally distributed. The log_volume however incorporates the seasonality. By observing the two peaks we can tell that more extreme values occur more frequently and that consequently the variable is not normally distributed (can be lead back to higher volume at beginning / end of trading day).
</p>
<p>
    To even out the intraday seasonality we must offset each log_volume against those that occur at the exact same time on different days. Hence, we must group the data by time of the day first and use this groups mean and standard deviation to perform a intraday z-normalization. By doing so, we receive a plot that nicely shows a normally distributed variable (code above, plot below).
</p>

In [ ]:
# Show histplot with curve of log returns and intraday z-normalized log volumes
## log_return
print("Log Return]")
plt.figure(figsize=(8, 4))
sns.histplot(df_stock_data['log_return'].dropna(), bins=50, kde=True)
plt.title('Histogramm of log returns')
plt.xlabel('log_return')
plt.xlim([-0.05, 0.05])
plt.ylabel('Occurences')
plt.show()

## log_volume
print("[Log Volume]")
plt.figure(figsize=(8, 4))
sns.histplot(df_stock_data['log_volume'].dropna(), bins=50, kde=True)
plt.title('Histogramm of log volumes')
plt.xlabel('log_volume')
plt.xlim([0, 17.5])
plt.ylabel('Occurences')
plt.show()

## log_volume_z_intraday
print("[z-Normalized Intraday Log Volume]")
plt.figure(figsize=(8, 4))
sns.histplot(df_stock_data['log_volume_z_intraday'].dropna(), bins=50, kde=True)
plt.title('Histogram of intraday z-normalized log volumes')
plt.xlabel('log_volume_z_intraday')
plt.xlim([-5, 5])
plt.ylabel('Occurences')
plt.show()



<p>
    <i>Implications of Normalization</i>
</p>
<p>
    In general, the z-transformation makes it possible to compare a single volume to all other volumes, independent of the time of the day. The intraday z-transformation however also takes the volatility of the time of the day into account and comares a single volume, that occurs at a certain time of the day, to all other volumes that occured at the same time of the day. This makes it possible to find effects beyond the temporal volatility. However, when interpreting the results, we must remember that each temporal group has its own standard deviation. 
</p>
<p>
    As the log_return already is approximately normally distributed, we do not need to normalize manually and can compute the metrics and compare it to the baseline (estimation window). 
    The log_volume still inherits the intraday effects however. To eliminate this bias, we must group the log volumes by the minute of the day and z-normalize based on this groups mean and standard deviation. By doing so, it is now more difficult to compute the CAV, AAV, and CAAV. More on this later...
</p>

<h1>Emotion Analysis - Part 3</h1>
<h2><i>Event Study</i></h2>
<h3>[3.1][Data Preparation]</h3>
<p>
    Grouping tweets to events and assigning a weighted event emotion.
</p>
<ul>
    <li>Filter stock data by market hours</li>
    <li>Group tweets with diff < 7h to an event</li>
    <li>Assign weighed event emotions based on engagement</li>
</ul>

In [ ]:
### Group tweets to events with dominant emotion ###

## Pre-process tweets for event study
# 1. Filter out tweets outside market hours (+-2h)
df_tweets_normalized = df_tweets_normalized[
    df_tweets_normalized['timestamp'].dt.time >= pd.to_datetime("04:00").time()
]
df_tweets_normalized = df_tweets_normalized[
    df_tweets_normalized['timestamp'].dt.time <= pd.to_datetime("17:59").time()
]

# 2. Sort by time
df_tweets_normalized = df_tweets_normalized.sort_values("timestamp").reset_index(drop=True)

# 3. Compute time difference between tweets
df_tweets_normalized['time_diff'] = df_tweets_normalized['timestamp'].diff()

# 4. Cluster Tweets to events with time_diff >= 7h
df_tweets_normalized['new_event'] = df_tweets_normalized['time_diff'] > pd.Timedelta(hours=7)

# 5. Cumulate events in event_id
df_tweets_normalized['event_id'] = df_tweets_normalized['new_event'].cumsum()

# 6. Compute the dominant emotion for each event and merge back to df
def compute_event_emotion(group):
    total_engagement = group['engagement'].sum()
    weighted_emo_scores = {
        emotion: (group[emotion] * group['engagement']).sum() / total_engagement
        for emotion in ekman_emotions
    }
    event_emotion = max(weighted_emo_scores, key=weighted_emo_scores.get)
    return pd.Series({
        **weighted_emo_scores,
        'event_emotion': event_emotion
    })

if not ('event_emotion' in df_tweets_normalized.columns):
    df_event_emotions = (df_tweets_normalized
        .drop(columns=['event_id'])
        .groupby(df_tweets_normalized['event_id'], group_keys=False)
        .apply(compute_event_emotion)
        .reset_index()
    )

    df_tweets_normalized = (df_tweets_normalized
        .merge(
            df_event_emotions[['event_id', 'event_emotion']],
            on='event_id',
            how='left'
        )
        .drop(columns={'anger', 'fear', 'joy', 'sadness', 'disgust', 'surprise', 'tweet_emotion', 'new_event'})
    )
df_tweets_normalized.head()

# 8. Set event from time to time of the earliest tweet (models from what point in time a reaction can occur)
df_tweets_normalized['event_time'] = df_tweets_normalized.groupby('event_id')['timestamp'].transform('min')

# 9. Filter duplicate events
df_tweets_normalized = df_tweets_normalized.drop_duplicates(subset=['event_id'])

print("[Info] Number of unique events = ", df_tweets_normalized['event_id'].nunique(), "\n")

In [ ]:
df_tweets_normalized.to_csv('../data/twitter/tweets_isTweet_emotions.csv', index=False)

<h3>[3.1][Create Event Study Dataframes]</h3>
<p>
    In this step, we eliminate some events that do not fulfill the following criteria. Hence, the number of unique events will be reduced. The data is filtered/processed as follows:
</p>
<ul>
    <li>Event timestamps are aligned with the subsequent stock data timestamp. Hence, they are rounded and assigned to the next full minute.</li>
    <li>Events that do not have a full 5h history of estimation trading data are sorted out.</li>
    <li>Events that do not have a full 2h post-event window of trading data ahead are sorted out.</li>
    <li>Events, whose estimation or event window exceed the trading day, are sorted out (in alignment to the previous two points).</li>
    <li>Additionally, the <u>abnormal_return</u> and <u>abnormal_volume</u> is computed (also intraday z-normalized)</li>
</ul>

In [ ]:
### Reading Event Study Dataframes ###
df_event_study = pd.read_csv('../data/twitter/emotion_event_study.csv')
df_event_study_pre_post = pd.read_csv('../data/twitter/emotion_event_study_pre_post.csv')

In [ ]:
### Create/Read Event Study DataFrame (with actual, expected, abnormal return and volume) ###

# 1. Filter events based on overlaps, estimation and observation windows
# 2. Calculate expected and abnormal returns for each event
def filter_events(df, get_pre = 0):
    # Define necessary vars
    est = 300
    obs = 120
    gap = est + obs
    results = []

    # 1. Filter by Overlaps
    valid_events = []
    last_event = None

    # Add valid events to the list
    for date in df.index:
        # First event always valid
        if last_event is None:
            valid_events.append(date)
            last_event = date
        else:
            # If timeshift > gaps, keep the current event and set as new last_event
            if (date - last_event) > pd.Timedelta(minutes=gap):
                valid_events.append(date)
                last_event = date

    # Only keep valid events
    df = df.loc[valid_events]
    print(f"[Overlap] New df_length = {len(df)}")

    # 2. Align event_time with the next available trading timestamp
    df.index = df.index.tz_convert(eastern)

    for i, date in enumerate(df.index):
        if date not in df_trades.index:
            future_times = df_trades.index[df_trades.index > date]
            if not future_times.empty:
                df.index.values[i] = pd.Timestamp(future_times[0])

    # 3. Filter events without sufficient estimation_window data
    # 4. Filter events without sufficient event_window data
    valid_events = []

    for date in df.index:
        day_data = df_trades[df_trades.index.date == date.date()]
        event_index = day_data.index.get_loc(date)

        remaining = len(day_data) - event_index

        if event_index >= est:
            if remaining >= obs:
                valid_events.append(date)

    df = df.loc[valid_events]
    print(f"[Sum Data] New df_length = {len(df)}")

    # 5. Event study variables
    for event_row in df.itertuples():
        event_id = event_row.event_id
        event_emo = event_row.event_emotion
        event_time = event_row.Index

        estimation_window = df_trades.loc[
            event_time - pd.Timedelta(minutes=est) : event_time - pd.Timedelta(minutes=1)
        ]
        event_window = df_trades.loc[
            event_time - pd.Timedelta(minutes=get_pre * est): event_time + pd.Timedelta(minutes=obs - 1)
        ]

        expected_return = estimation_window['log_return'].mean()
        expected_volume = estimation_window['log_volume'].median() # robust against outliers, see Bamber (1987)
        expected_volume_z_intraday = estimation_window['log_volume_z_intraday'].median()

        for i, fin_row in enumerate(event_window.itertuples()):
            if get_pre == 1:
                i = int((fin_row.Index - event_time).total_seconds() // 60)

            actual_return = fin_row.log_return
            abnormal_return = actual_return - expected_return

            actual_volume = fin_row.log_volume
            abnormal_volume = actual_volume - expected_volume

            actual_volume_z_intraday = fin_row.log_volume_z_intraday
            abnormal_volume_z_intraday = actual_volume_z_intraday - expected_volume_z_intraday

            results.append({
                'event_id': event_id,
                'event_fin_offset': i,
                'event_time': event_time,
                'fin_time': fin_row.Index,
                'event_emo': event_emo,
                'actual_return': actual_return,
                'expected_return': expected_return,
                'abnormal_return': abnormal_return,
                'actual_volume': actual_volume,
                'expected_volume': expected_volume,
                'abnormal_volume': abnormal_volume,
                'actual_volume_z_intraday': actual_volume_z_intraday,
                'expected_volume_z_intraday': expected_volume_z_intraday,
                'abnormal_volume_z_intraday': abnormal_volume_z_intraday
            })

    # Make a dataframe from the result list
    df_valid_events = pd.DataFrame(results)

    return df_valid_events

In [ ]:
if RUN_TYPE == 1: # if not yet created, create the event study dataframes with abnormal returns and volumes

    #  Create new dfs with necessary event data and set time column as index
    df_events = (df_tweets_normalized[['event_id', 'event_emotion', 'event_time']]
                .set_index('event_time')
                .sort_index())
    df_trades = (df_stock_data[['timestamp', 'log_return', 'log_volume', 'log_volume_z_intraday']]
                .set_index('timestamp')
                .sort_index())

    df_event_study = filter_events(df_events, get_pre = 0).to_csv('../data/twitter/emotion_event_study.csv', index=False)
    df_event_study_pre_post = filter_events(df_events, get_pre=1).to_csv('../data/twitter/emotion_event_study_pre_post.csv', index=False)

print("[Info] Dataframe Inspection")
print(f"Emotions: {df_event_study['event_emo'].unique()}")
emotion = df_event_study['event_emo'].unique()
for emo in emotion:
    print(f"  - {emo}: {len(df_event_study[df_event_study['event_emo'] == emo].groupby('event_id'))} events")

print(f"\n[Info] Event Study DataFrame has {len(df_event_study)} entries.")
print(f"[Info] Event Study Pre/Post DataFrame has {len(df_event_study_pre_post)} entries.\n")
if (df_event_study_pre_post.columns == df_event_study.columns).all():
    print("[Info] emotion_event_study(_pre_post).csv files have the following columns:")
    print(df_event_study.columns.tolist())

<p><i>Resulting Dataframe</i></p>
<p>
    We get a dataframe that contains all events with their corresponding event_window (and estimation_window) tuples, containing offset and abnormal returns / volumes. With this, we can now compute our event study metrics.
</p>

<h3>[3.2][Event Study on Return]</h3>


<h4><u>3.2.1 Abnormal Return</u></h4>
<p>[Intro]</p>
<p>In the following it is analysed to what extent a significant effect of a particular emotion on the change in stock return can be observed by the minute<br>
--> Is the mean abnormal return significantly greater / lower than 0 resp. does the mean abnormal return significantly differ from the expected return?</p>
<p>
    <ul>
        <li>H0: mean abnormal return = 0</li>
        <li>H1: mean abnormal return >< 0</li>
    </ul>
</p>

In [ ]:
### Significance Testing of AR ###

print(f"[Info] Testing for significant effect of emotions {df_event_study['event_emo'].unique()} on abnormal returns ...")

## Functions
def stars(p):
    if p < 0.001: return '***'
    elif p < 0.01: return '**'
    elif p < 0.05: return '*'
    else: return ''
    
def test_abnormal_returns(df, test_col):
    results = []

    emotions = df['event_emo'].unique()
    minutes = df['event_fin_offset'].unique()

    for minute in minutes:
        row = {'minute': minute}

        for emo in emotions:
            values = df.loc[
                (df['event_fin_offset'] == minute) & (df['event_emo'] == emo),
                test_col
            ]
            if len(values) > 1:
                t_stat, p_value = stats.ttest_1samp(values, 0)
                row[f'{emo}_{test_col}_mean'] = values.mean()
                row[f'{emo}_{test_col}_t_stat'] = t_stat
                row[f'{emo}_{test_col}_p_value'] = p_value
                row[f'{emo}_{test_col}_sig'] = f"{t_stat:.2f}{stars(p_value)}" # adds stars 
            else:
                row[f'{emo}_{test_col}_mean'] = np.nan
                row[f'{emo}_{test_col}_t_stat'] = np.nan
                row[f'{emo}_{test_col}_p_value'] = np.nan
                row[f'{emo}_{test_col}_sig'] = ''
        
        results.append(row)
    return pd.DataFrame(results)

## Test abnormal returns and visualize results
# Test the abnormal returns for each emotion and minute
ar_test_scores = test_abnormal_returns(df_event_study, 'abnormal_return')
ar_test_scores.set_index('minute', inplace=True)

# Show only rows with significant p-values
alpha = 0.05

# Make list of p-value columns
pval_cols = [col for col in ar_test_scores.columns if col.endswith('_p_value')]

# Keep only rows where at least one p-value is below alpha
significant_rows = ar_test_scores[(ar_test_scores[pval_cols] < alpha).any(axis=1)]

# Show rows that contain a significant p-value
sig_cols = [col for col in significant_rows.columns if col.endswith('_sig')]
significant_rows[sig_cols]

In [ ]:
### Plot Heatmap of t-Statistics for Abnormal Returns ###

# Emotionen extrahieren (alle Spalten mit '_t_stat' am Ende)
emotions = [col.replace('_t_stat', '') for col in ar_test_scores.columns if col.endswith('_t_stat')]

# DataFrame für Heatmap vorbereiten: Zeilen = Emotionen, Spalten = Minuten
heatmap_data = pd.DataFrame({
    emo: ar_test_scores[f"{emo}_t_stat"] for emo in emotions
}).T  # Transponieren: Emotionen als Zeilen

plt.figure(figsize=(16, 6))
sns.heatmap(
    heatmap_data,
    cmap="coolwarm",
    center=0,
    annot=False,
    cbar_kws={'label': 't-Statistik'}
)
plt.xlabel("Minute nach Event")
plt.ylabel("Emotion")
plt.title("Heatmap der t-Statistiken der abnormal returns pro Emotion und Minute")
plt.tight_layout()
plt.show()

<p>[Test-Results]</p>
<p><i>What was tested?</i></p>
<p>For each emotion and each minute in the post-event window it was tested if the mean abnormal return significantly deviates from 0. The goal was to detect whether there is a deviation from the expected return (H0: mean = 0).</p>
<p><i>Results</i></p>
<p>The heatmap shows that only for few points across the emotions, a significant effect could be observed, where the mean abnormal return significantly deviates from 0. Thus, the H1: mean >< 0 can most likely be refused.
    <ul>
        <li>No consistent pattern of significant observations visible, that may indicate a recurrent effect.</li>
        <li>There are a few significant events across the heatmap. Because of their low density they can probably be declared as outliers.</li>
    </ul>
</p>
<p><i>Conclusion</i></p>
<p>In the short-term and for individual points in time, an impact of a tweets emotion on the expected return of the stock can be neglected.</p>

<h4><u>3.2.2 Cumulative Abnormal Return</u></h4>
<p>
    [Intro]
</p>
<p>
    In the following it is analysed, whether an effect occurs within specified time windows. As log returns are additive, we can use the cumsum() Method to compute the cumulative sum and plot it over time. This way, we can yield a graph that shows the actual return over time and how the abnormal return deviates after the event.
</p>

In [ ]:
### [All emotions] Plotting Cumulative Log Returns Around Tweets by emotion ###

# Present emotions in the DataFrame
emotions = df_event_study_pre_post['event_emo'].unique()

for emo, i in zip(emotions, range(len(emotions))):

    # 0. Windows
    estimation_window = df_event_study_pre_post
    event_window = df_event_study_pre_post[df_event_study_pre_post['event_fin_offset'] >= 0]

    # 0.5 Plot Overall Cumulative Log Returns Around Tweets 
    if i == 0:
        actual_returns = estimation_window.groupby('event_fin_offset')['actual_return'].mean()
        cumulative_actual_return = actual_returns.cumsum()

        abnormal_returns = event_window.groupby('event_fin_offset')['abnormal_return'].mean()
        cumulative_abnormal_return = abnormal_returns.cumsum()

        print("[Overall]")
        plt.figure(figsize=(10, 5))
        plt.plot(cumulative_actual_return.index, cumulative_actual_return.values * 100, label="Actual Log Return (%)", color="blue")
        plt.plot(cumulative_abnormal_return.index, cumulative_abnormal_return.values * 100, label="Abnormal Log Return (%)", color="orange")
        plt.axvline(0, linestyle="--", color="black", label="Tweet")
        plt.axhline(0, linestyle="-", color="gray")
        plt.title("Cumulative Log Returns Around Tesla Tweets (Overall)")
        plt.xlabel("Minute Offset from Tweet")
        plt.xticks([-300, -240, -180, -120, -60, 0, 60, 120])
        plt.ylabel("Cumulative Return (%)")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()
        print("-----------------------------------------------------------------------------------------------------------------\n")
        print("[Emotions]\n")

    # 1. Calculate cumulative actual log returns around tweets
    actual_returns = estimation_window[estimation_window['event_emo'] == emo].groupby('event_fin_offset')['actual_return'].mean()
    cumulative_actual_return = actual_returns.cumsum()

    # 2. Caluculate cumulative abnormal log returns around tweets
    abnormal_returns = event_window[event_window['event_emo'] == emo].groupby('event_fin_offset')['abnormal_return'].mean()
    cumulative_abnormal_return = abnormal_returns.cumsum()

    # 3. Plot
    # Print CAR over the event window
    caar = abnormal_returns.sum()  # Sum of mean abnormal returns over the event window (caar)
    print(f"[Plot] Plotting cumulative log returns for emotion: {emo}")
    print(f"[Info] Cumulative Average Abnormal Return (CAAR) for {emo} over the event window: {caar* 100:.4f} %\n")
    plt.figure(figsize=(10, 5))
    plt.plot(cumulative_actual_return.index, cumulative_actual_return.values * 100, label="Actual Log Return (%)", color="blue")
    plt.plot(cumulative_abnormal_return.index, cumulative_abnormal_return.values * 100, label="Abnormal Log Return (%)", color="orange")
    plt.axvline(0, linestyle="--", color="black", label="Tweet")
    plt.axhline(0, linestyle="-", color="gray")
    plt.title("Cumulative Log Returns Around Tesla Tweets (Emotion: " + emo + ")")
    plt.xlabel("Minute Offset from Tweet")
    plt.xticks([-300, -240, -180, -120, -60, 0, 60, 120])
    plt.ylabel("Cumulative Return (%)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()



In [ ]:
### Test for significant effect of emotions on abnormal volume in window ###

windows = [(0, 1), (0, 2), (0, 5), (0, 10), (0, 30), (0, 60), (0, 120)]

results = []
for emo, i in zip(emotions, range(len(emotions))):
    # Iterate through each time window and test for significant effects
    for start, end in windows:

        # Here: testing overall effect 
        if i == 0:
            # Filter time window without emotion
            window = df_event_study[
                (df_event_study['event_fin_offset'] >= start) &
                (df_event_study['event_fin_offset'] <= end)
            ]    
            
            # Compute CAR for each event
            event_car = window.groupby('event_time')['abnormal_return'].sum()

            # Calculate mean CAR across all events
            mean_car = event_car.mean() 

            # Perform t-test against zero (significantly different from zero)
            t_stat, p_value = stats.ttest_1samp(event_car, 0)

            results.append({
                'emotion': 'NaN',
                'window': f"[{start}, {end}]",
                'mean_car': f"{mean_car * 100:.4f} %",  # Convert to percentage
                't_stat': f"{round(t_stat, 2)}{stars(p_value)}",  # Add stars for significance
            })

        # Here: testing for specific emotion
        else:   
            # Filter for the current emotion and time window
            emo_window = df_event_study[
                (df_event_study['event_emo'] == emo) &
                (df_event_study['event_fin_offset'] >= start) &
                (df_event_study['event_fin_offset'] <= end)
            ]
        
            # Compute CAR for each event
            event_car = emo_window.groupby('event_time')['abnormal_return'].sum()

            # Calculate mean CAR across all events
            mean_car = event_car.mean() 

            # Perform t-test against zero (significantly different from zero)
            t_stat, p_value = stats.ttest_1samp(event_car, 0)

            results.append({
                'emotion': emo,
                'window': f"[{start}, {end}]",
                'mean_car': f"{mean_car * 100:.4f} %",  # Convert to percentage
                't_stat': f"{round(t_stat, 2)}{stars(p_value)}",  # Add stars for significance
            })

df_car_results = pd.DataFrame(results)

for emo in df_car_results['emotion'].unique():
    print(f"\n[Info] Results for emotion '{emo}':")
    emo_results = df_car_results[df_car_results['emotion'] == emo]
    print(emo_results[['window', 'mean_car', 't_stat']])

<p>[Test-Results]</p>
<p><i>What was tested?</i></p>
<p>For each emotion and each minute in the post-event window it was tested if the mean abnormal return significantly deviates from 0. The goal was to detect whether there is a deviation from the expected return (H0: mean = 0).</p>
<p><i>Results</i></p>
<p>The heatmap shows that only for few points across the emotions, a significant effect could be observed, where the mean abnormal return significantly deviates from 0. Thus, the H1: mean >< 0 can most likely be refused.
    <ul>
        <li>No consistent pattern of significant observations visible, that may indicate a recurrent effect.</li>
        <li>There are a few significant events across the heatmap. Because of their low density they can probably be declared as outliers.</li>
    </ul>
</p>
<p><i>Conclusion</i></p>
<p>In the short-term and for individual points in time, an impact of a tweets emotion on the expected return of the stock can be neglected.</p>

<h3>[3.3][Event Study on Volume]</h3>
<p><i>WILL BE PERFORMED LATER IF THERE IS STILL TIME</i></p>

<h4><u>3.3.1 Abnormal Volume</u></h4>
<p>In the following it is analysed to what extent a significant effect of a particular emotion on the change in trade volume can be observed by the minute<br>
--> Is the mean abnormal volume significantly greater / lower than 0 resp. does the mean abnormal volume significantly differ from the expected volume?</p>
<p>
    <ul>
        <li>H0: mean abnormal volume = 0</li>
        <li>H1: mean abnormal volume >< 0</li>
    </ul>
</p>
<p>Here, the absolute trading volume is logarithmized for each offset. However, as volume is interesting as an absolute value (how many stocks where traded), the value is always positive and may deviate a lot. Hence, the log_volume is not normally distributed (see Illustration below) and a t-test cannot be performed as it assumes a normal distribution of the variable.</p>
<p>Consequently, a z-transformation is performed on the log_volume to be able to properly test for significance.</p>